In [1]:
# pd.value_counts(df_train_transform.Collision_Severity, normalize = True)

In [2]:
#  from sklearn.metrics import accuracy_score,f1_score
# majority_class = df_train_transform.Collision_Severity.mode()[0]
# # y_pred = np.full(shape = df_train_transform.Collision_Severity.shape, fill_value = majority_class)
# accuracy_score(df_train_transform.Collision_Severity, y_pred)

In [3]:
# from sklearn.metrics import roc_auc_score
# roc_auc_score(df_train_transform.Collision_Severity, y_pred)

In [1]:
import pandas as pd
from sklearn.base import TransformerMixin
import numpy as np
import plotly.graph_objects as go
import numpy as np

In [2]:
df_train = pd.read_csv("Accident_train.csv")
df_test = pd.read_csv("Accident_test.csv")

In [3]:
print('Train : Records:', df_train.shape[0], '\nColumns:', df_train.shape[1])
print()

print('Test  :  Records:', df_test.shape[0], '\nColumns:', df_test.shape[1])

Train : Records: 10043 
Columns: 17

Test  :  Records: 354 
Columns: 17


In [4]:
print('Proportion of Missing Values in Training Dataset:', 
      round(df_train.isna().sum().sum()/len(df_train),3), '%')

print('Proportion of Missing Values in Testing Dataset:', 
      round(df_test.isna().sum().sum()/len(df_test),3), '%')

Proportion of Missing Values in Training Dataset: 0.242 %
Proportion of Missing Values in Testing Dataset: 0.277 %


In [5]:
train_values=df_train.values
test_values=df_test.values

In [6]:
X_train=pd.DataFrame(train_values)
X_test=pd.DataFrame(test_values)

In [7]:
class DataFrameImputer(TransformerMixin):

    def __init__(self):
        """Impute missing values.
        Columns of dtype object are imputed with the most frequent value 
        in column.

        Columns of other types are imputed with mean of column.

        """
    def fit(self, X, y=None):

        self.fill = pd.Series([X[c].value_counts().index[0]
            if X[c].dtype == np.dtype('O') else X[c].mean() for c in X],
            index=X.columns)

        return self

    def transform(self, X, y=None):
        return X.fillna(self.fill)

In [8]:
df_train_transform = DataFrameImputer().fit_transform(X_train)
df_test_transform  = DataFrameImputer().fit_transform(X_test)

In [9]:
df_train_transform.columns=['Collision_Ref_No', 'Policing_Area', 'Collision_Severity', 'Weekday_of_Collision', 'Day_of_Collision',
                            'Month_of_Collision', 'Hour_of_Collision', 'Carriageway_Type', 'Speed_Limit', 'Junction_Detail', 
                            'Junction_Control', 'Ped_Crossing_HC', 'Ped_Crossing_PC', 'Light_Conditions', 'Weather_Conditions',
                            'Road_Surface_Conditions', 'Special_Conditions_at_Site']

In [10]:
df_test_transform.columns=['Collision_Ref_No', 'Policing_Area', 'Collision_Severity', 'Weekday_of_Collision', 'Day_of_Collision',
                            'Month_of_Collision', 'Hour_of_Collision', 'Carriageway_Type', 'Speed_Limit', 'Junction_Detail', 
                            'Junction_Control', 'Ped_Crossing_HC', 'Ped_Crossing_PC', 'Light_Conditions', 'Weather_Conditions',
                            'Road_Surface_Conditions', 'Special_Conditions_at_Site']


In [11]:
week_days = {'FRI': 5, 'MON': 1, 'SAT': 6, 'SUN': 7, 'THU': 4, 'TUE': 2, 'WED': 3}
df_train_transform['Weekday_of_Collision'] = [week_days[day] for day in df_train_transform['Weekday_of_Collision']]

In [32]:
y = df_train_transform['Collision_Severity']
X = df_train_transform.drop(['Collision_Severity', 'Policing_Area'], axis=1)

In [33]:
from sklearn.model_selection import train_test_split
# from imblearn.over_sampling import SMOTE
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=27)
# sm = SMOTE(random_state=27)
# X_train, y_train = sm.fit_sample(X_train, y_train)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=27)

In [34]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

smote = LogisticRegression(solver='liblinear').fit(X_train, y_train)

smote_pred = smote.predict(X_test)

# Checking accuracy
accuracy_score(y_test, smote_pred)

0.8916766228594185

In [35]:
print(len(X_train.columns))
X_train.columns

15


Index(['Collision_Ref_No', 'Weekday_of_Collision', 'Day_of_Collision',
       'Month_of_Collision', 'Hour_of_Collision', 'Carriageway_Type',
       'Speed_Limit', 'Junction_Detail', 'Junction_Control', 'Ped_Crossing_HC',
       'Ped_Crossing_PC', 'Light_Conditions', 'Weather_Conditions',
       'Road_Surface_Conditions', 'Special_Conditions_at_Site'],
      dtype='object')

In [36]:
from sklearn.utils import resample
X = pd.concat([X_train, y_train], axis=1)
X.Collision_Severity.value_counts()

3    6725
2     716
1      91
Name: Collision_Severity, dtype: int64

In [37]:
fatal = X[X.Collision_Severity==1]
serious = X[X.Collision_Severity==2]
slight = X[X.Collision_Severity==3]

In [38]:
# upsample minority
fatal_upsampled = resample(fatal,
                          replace=True, # sample with replacement
                          n_samples=len(slight), # match number in majority class
                          random_state=27) # reproducible results
serious_upsampled = resample(serious,
                          replace=True, # sample with replacement
                          n_samples=len(slight), # match number in majority class
                          random_state=27) # reproducible results


# combine majority and upsampled minority
upsampled = pd.concat([fatal_upsampled, serious_upsampled, slight])

# check new class counts
upsampled.Collision_Severity.value_counts()

3    6725
2    6725
1    6725
Name: Collision_Severity, dtype: int64

In [39]:
X_train.columns

Index(['Collision_Ref_No', 'Weekday_of_Collision', 'Day_of_Collision',
       'Month_of_Collision', 'Hour_of_Collision', 'Carriageway_Type',
       'Speed_Limit', 'Junction_Detail', 'Junction_Control', 'Ped_Crossing_HC',
       'Ped_Crossing_PC', 'Light_Conditions', 'Weather_Conditions',
       'Road_Surface_Conditions', 'Special_Conditions_at_Site'],
      dtype='object')

In [42]:
y_train = upsampled.Collision_Severity
X_train = upsampled.drop(['Collision_Severity'], axis=1)
upsampled = LogisticRegression(solver='liblinear').fit(X_train, y_train)

upsampled_pred = upsampled.predict(X_test)

In [43]:
accuracy_score(y_test, upsampled_pred)

0.5117483074472322

In [44]:
upsampled=LogisticRegression().fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS EXCEEDS LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression



In [45]:
# upsample minority
fatal_upsampled = resample(fatal,
                          replace=True, # sample with replacement
                          n_samples=len(slight), # match number in majority class
                          random_state=27) # reproducible results
serious_upsampled = resample(serious,
                          replace=True, # sample with replacement
                          n_samples=len(slight), # match number in majority class
                          random_state=27) # reproducible results


# combine majority and upsampled minority
upsampled = pd.concat([fatal_upsampled, serious_upsampled, slight])

# check new class counts
upsampled.Collision_Severity.value_counts()

3    6725
2    6725
1    6725
Name: Collision_Severity, dtype: int64

In [46]:
X_train.columns

Index(['Collision_Ref_No', 'Weekday_of_Collision', 'Day_of_Collision',
       'Month_of_Collision', 'Hour_of_Collision', 'Carriageway_Type',
       'Speed_Limit', 'Junction_Detail', 'Junction_Control', 'Ped_Crossing_HC',
       'Ped_Crossing_PC', 'Light_Conditions', 'Weather_Conditions',
       'Road_Surface_Conditions', 'Special_Conditions_at_Site'],
      dtype='object')

In [47]:
y_train = upsampled.Collision_Severity
X_train = upsampled.drop(['Collision_Severity'], axis=1)

upsampled = LogisticRegression(solver='liblinear').fit(X_train, y_train)

upsampled_pred = upsampled.predict(X_test)

In [48]:
accuracy_score(y_test, upsampled_pred)

0.5117483074472322

In [49]:
upsampled=LogisticRegression().fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS EXCEEDS LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression



In [50]:
upsampled_pred = upsampled.predict(X_test)

In [51]:
accuracy_score(y_test, upsampled_pred)

0.5487853444842692

In [52]:
from sklearn import metrics

In [53]:
print(metrics.classification_report( y_test, upsampled_pred, digits=3))

              precision    recall  f1-score   support

           1      0.028     0.483     0.052        29
           2      0.144     0.395     0.211       243
           3      0.946     0.566     0.708      2239

    accuracy                          0.549      2511
   macro avg      0.373     0.481     0.324      2511
weighted avg      0.857     0.549     0.653      2511

